## 날씨 현황을 알려주는 메신져봇 구현

In [1]:
import telegram

In [2]:
# 텔레그램 API 키 등록
with open('telegram_token.txt') as f:
    mytoken = f.read()
mytoken

'6540124535:AAFlmA7M3sEEaHtXK451bGTofe1dyEJ7miU'

In [3]:
# 텔레그램 봇 초기화
bot = telegram.Bot(token=mytoken)

In [4]:
from telegram.ext import Updater
from telegram.ext import MessageHandler
from telegram.ext import CommandHandler
from telegram.ext import Filters

### 날씨에 따른 이모지 출력함수

In [5]:
def wfgrade(wf):
    emoji = '🌞'
    if wf == '구름많음': emoji = '☁️'
    elif wf == '흐림': emoji = '😶‍🌫️'
    elif wf == '흐리고 비': emoji = '☔'
    elif wf == '흐리고 눈': emoji = '☃️'
    return emoji

### 날씨 정보 수집하고 조회하는 함수

In [6]:
import requests
from bs4 import BeautifulSoup

In [7]:
url = 'https://www.kma.go.kr/weather/forecast/mid-term-rss3.jsp?stnId=109'
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36'}

In [8]:
# 날씨 데이터를 내려받아 xml파일에 저장
def reload_weather():
    res = requests.get(url, headers=headers)
    with open('./data/weather.xml', 'w', encoding='UTF-8') as f:
        f.writelines(res.text)

reload_weather()

In [9]:
# 특정 지역의 날씨를 조회해서 결과 반환
def fetch_weather(region):
    with open('./data/weather.xml', encoding='UTF-8') as f:
        wtdata = BeautifulSoup(f.read(), 'xml')
        for loc in wtdata.findAll('location'):
            if loc.city.text == region:
                for data in loc.findAll('data'):
                    wtinfo = f'{loc.city.text}, {data.tmEf.text},\n'\
                          f'{data.wf.text} {wfgrade(data.wf.text)}, '\
                          f'{data.tmx.text}, '\
                          f'{data.tmx.text}, {data.rnSt.text}'
                    return wtinfo

fetch_weather('서울')

'서울, 2023-09-23 00:00,\n구름많음 ☁️, 27, 27, 30'

### 텔레그램 챗봇 응답처리 콜백함수

In [10]:
# 날씨 정보 출력
def weather_message(update, context):
    wtinfo = fetch_weather(update.message.text)
    context.bot.send_message(update.effective_chat.id, wtinfo)

In [11]:
# 날씨 정보 갱신
def reload_message(update, context):
    reload_weather()
    context.bot.send_message(update.effective_chat.id, '💫날씨정보 갱신됨!!💫')

### 텔레그램 챗봇 서버 초기화

In [12]:
print('텔레그램 RPA 서버 준비중...')
updater = Updater(mytoken, use_context=True)

텔레그램 RPA 서버 준비중...


#### 콜백함수 등록

In [13]:
weather_handler = MessageHandler(Filters.text, weather_message)
reload_handler = CommandHandler('reload', reload_message)

#### 핸들러 함수 등록

In [14]:
updater.dispatcher.add_handler(reload_handler)
updater.dispatcher.add_handler(weather_handler)

#### 텔레그램 챗봇 서버 실행

In [15]:
print('텔레그램 RPA 서버 실행중...')
updater.start_polling(timeout=3, drop_pending_updates=True)
updater.idle()

텔레그램 RPA 서버 실행중...


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "/opt/conda/lib/python3.11/site-packages/telegram/ext/dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "/opt/conda/lib/python3.11/site-packages/telegram/ext/handler.py", line 198, in handle_update
    return self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_28993/684099546.py", line 4, in weather_message
    context.bot.send_message(update.effective_chat.id, wtinfo)
  File "/opt/conda/lib/python3.11/site-packages/telegram/bot.py", line 130, in decorator
    result = func(*args, **kwargs)
             ^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/telegram/bot.py", line 513, in send_message
    return self._message(  # type: ignore[return-value]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/conda/lib/python3.11/site-packages/telegram/ext/extbot.p